# Deep learning for computer vision
This section covers convolutional neural networks (*convnets*), which is a type of deep learning model that is almost universally used in computer vision applications.

## 5.1 Introduction to convnets
Let's start by taking a practical look at a simple convnet example used to classify MNIST digits. In a previous model, we used a densely connected network that achieved a test accuracy of 97.8%. Let's see how this convnet model does on the same task:

In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


For its input a convnet takes input tensors of shape (`image_height, image_width, image_channels`). In this case, we have images that are 28x28 pixels, and only a black/white channel.

Let's view the architecture of the convnet so far:

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Interestingly, the width and height dimensions tend to shrink as you go deeper into the network. 

The next step is to feed the last output tensor (shape (3, 3, 64)) into a densely connected classifier network: a stack of `Dense` layers. These classifiers process 1D vectors, whereas the current output is a 3D tensor. To convert the 3D tensors into 1D vectors, we need to flatten them, and then add a few `Dense` layers on top.

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

We'll do 10-way classification by using a final layers with 10 outputs and a softmax activation. Here is what the network looks like so far:

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

As we can see, the (3, 3, 64) output tensors are flattened into vectors of shape (576,) before doing into two `Dense` layers.

Now, let's train the convnet on the MNIST digits.

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [7]:
# Change the format of the training & testing images
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Change dtype and convert to a 0-1 scale
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Set labels to categorical one-hot-encoded
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [9]:
# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# fit the model to training data
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 79s - loss: 0.1729 - acc: 0.9456    
Epoch 2/5
60000/60000 [==============================] - 81s - loss: 0.0477 - acc: 0.9847    
Epoch 3/5
60000/60000 [==============================] - 80s - loss: 0.0322 - acc: 0.9901    
Epoch 4/5
60000/60000 [==============================] - 88s - loss: 0.0251 - acc: 0.9925    - E
Epoch 5/5
60000/60000 [==============================] - 84s - loss: 0.0189 - acc: 0.9942    


In [10]:
# Evaluate model on test data
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 5s     


0.99150000000000005

The densely connected network from a previous example had a test accuracy of 97.8%, but the basic convnet has a test accuracy of **99.2%**! Great improvement!

To get deeper into the topic, we must ask why this simple convnet works so well, compared to a densely connected model? Let's dive into what the `Conv2D` and `MaxPooling2D` layers actually do.

### 5.1.1 The convolution operation
The core difference between a densely connected layer and a convolutional layer is this: `Dense` layers learn *global* patterns in their input feature space, whereas convolutional layers learn *local* patterns. In this case of images, those patterns are found in small 2D windows of the inputs shown below:

![cnn_digit](images/5_1_1_cnn_digit.jpg)

This key characteristic gives convnets two interesting properties:
 - *The patterns they learn are translation invariant*. After learning a pattern in any part of the picture, a convnet can recognize it anywhere. A densely connected network would have to learn the pattern over again if it appeared in a new location. This enables convnets to be trained on fewer samples to learn representations that have generalization power.
 - *They can learn spatial hierarchies of patterns*. A first convolutional layer will learn small local patterns such as edges, a secons convolutional layer will learn larger patterns made of the features of the first layers, and so on. This allows convnets to efficiently learn increasingly complex and abstract visual concepts.
 
![cat](images/5_1_1_cat.jpg)

Convolutions operate over 3D tensors, called **feature maps**, with height, width, and depth (channels) axes. For an RGB image, the dimension of the depth axis is 3, because the image has three color channels: red, green, and blue. The convolution extracts patches from its input feature map and applies the same transformation to all of these patches, producing an **output feature map**. The different channels of the output depth axis no longer stand for specific colors is in RGB, but they stand for *filters*. Filters encode specific aspects of the input data, such as presence of a face.

In the MNIST example, the first convolutional layer takes a (28, 28, 1) sized feature map and outputs a feature map of size (26, 26, 32). It computes 32 filters over its input. Each of the 32 output channels contains a 26x26 grid of values, which is a *response map* of the filter over the input, indicating the response of that filter pattern at different locations in the input. Feature map means: every dimension in the depth axis is a feature (or filter), and the 2D tensor output is the 2D spatial map of the response of this filter over the input. The image below shows a single filter that detects for diagonal edges.

![response_map](images/5_1_1_responsemap.jpg)

Convolutions are defined by two key parameters:
 - **Size of the patches extracted from the inputs**- Typically 3x3 or 5x5.
 - **Depth of the output feature map**- Number of filters computed by the convolution.
 
In Keras `Conv2D` layers, these parameters are the first arguments passed to the layer:

`Conv2D(output_depth, (window_height, window_width))`

A convolution works by sliding these 3x3 (or 5x5) windows over the 3D input feature map, stopping at every possible location and extracting the 3D patch of surrounding features (`window_height, window_width, input_depth`). Each 3D patch is transformed into a 1D vector (`output_depth,`). All these vectors are then spatially reassembled into a 3D output map of shape (`height, width, output_depth`). Every spatial location in the output feature map corresponds to the same location in the input feature map. 

![convolution](images/5_1_1_convolution.jpg)

The output width and height may differ from the input width and height because can be attributed to border effects, or the use of *strides*.

Let's take a deeper look at this once more:

**UNDERSTANDING BORDER EFFECTS AND PADDING**
Consider a 5x5 feature map. There are only 9 tiles which you can center a 3x3 window, forming a 3x3 grid. Hence, the output feature map will be 3x3. 

![patch](images/5_1_1_patch.jpg)

If you want an output feature map with the same spatial dimensions as the input, you can use **padding**. Padding consists of adding an appropriate number of rows and columns on each side of the input feature map so you can fit center convolution windows around every input tile.

![padding](images/5_1_1_padding.jpg)

**UNDERSTANDING CONVOLUTION STRIDES**
The other factor that can influence output size is the notion of **strides**. So far, we have described convolutions assuming the center tiles of the convolution windows are all contiguous. The distance between two successive windows is a parameter of the convolution, called the stride, with a default value of 1. It's possible to have *strided convolutions* with a stride greater than 1. The image below shows a convolution with a stride of 2.

![stride](images/5_1_1_stride.jpg)

Using stride 2 means the width and height of the feature map are downsampled by a factor of 2. Strided convolutions are rarely used in practice, however. To downsample feature maps, we instead tend to use **max-pooling**.

### 5.1.2 The max-pooling operation
The role of max pooling is to aggressively downsample feature maps, much like strided convolutions. Max pooling consists of extracting windows from the input feature maps and outputting the max value of each channel. This is similar to convolution, but instead of transforming local patches via a learned linear transformation, they are transformed via a hardcoded max tensor operation. Max pooling is usually done with 2x2 windows and stride 2, in order to downsample the feature maps by a factor of 2. Conversely, convolution is typically done with a 3x3 window and a stride of 1.

Let's view this is coded context:

In [11]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64, (3,3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3,3), activation='relu'))

In [12]:
model_no_max_pool.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


What's wrong with this setup? Two things:
 - It doesn't allow a learning of spatial hierarchy of features. The 3x3 windows in the third layer will only contain information coming from 7x7 windows in the initial input. The high-level patterns learned by the convnet will still be very small with regard to the initial input, which may not be enough to learn to classify digits. We need features from the last convolution layer to contain information about the totality of the input.
 - The final feature map has 22x22x64 = 30,976 total coefficients per sample. This is huge. If you were to flatten it to stick a `Dense` layer of size 512 on top, that layer would have 15.8 million parameters. This is far too large for such a small model and would result in intense overfitting.
 
The reason to use downsampling is to reduce the number of feature-map coefficients to process, as well as induce spatial-filter hierarchies by making successive convolution layers look at increasingly large windows.

Max pooling tends to work better than most alternatives because features tend to encode the spatial presence of some pattern or concept over the different tiles of the feature map, and it's more informative to look at the maximal presence of different features than at their average presence. The most reasonable subsampling strategy is to first produce dense maps of features and then look at the maximal activation of the features over small patches, rather than looking at sparser windows of the inputs (strided convolutions), which could cause you to miss or dilute feature-presence information.